<a href="https://colab.research.google.com/github/parkerburchett/NumeraiScripts/blob/main/Load_new_data_into_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# source https://github.com/numerai/example-scripts/blob/master/example_model.py

!pip install numerapi
!pip install halo

  Created wheel for halo: filename=halo-0.0.31-py3-none-any.whl size=11260 sha256=af6dfef775f00291e81a661aa393f5535dcfb2235a036246e92a50b5d9c6928e
  Stored in directory: /root/.cache/pip/wheels/95/ff/20/5d16a0059f20c5e60be2df845201e73af179a5a79a3d566f48
Successfully built halo


In [22]:
import pandas as pd
import gc
import numerapi
import pandas as pd
from halo import Halo


napi = numerapi.NumerAPI(verbosity="info")
spinner = Halo(text='', spinner='dots')

current_round = napi.get_current_round(tournament=8)  # tournament 8 is the primary Numerai Tournament

# read in all of the new datas
# tournament data and example predictions change every week so we specify the round in their names
# training and validation data only change periodically, so no need to download them over again every single week
napi.download_dataset('numerai_training_data.parquet', 'numerai_training_data.parquet')
napi.download_dataset('numerai_tournament_data.parquet', f'numerai_tournament_data_{current_round}.parquet')
napi.download_dataset('numerai_validation_data.parquet', 'numerai_validation_data.parquet')
napi.download_dataset('example_predictions.parquet', f'example_predictions_{current_round}.parquet')
napi.download_dataset('example_validation_predictions.parquet', f'example_validation_predictions.parquet')

spinner.start('Reading parquet data')
training_data = pd.read_parquet('numerai_training_data.parquet')
tournament_data = pd.read_parquet(f'numerai_tournament_data_{current_round}.parquet')
validation_data = pd.read_parquet('numerai_validation_data.parquet')
example_preds = pd.read_parquet(f'example_predictions_{current_round}.parquet')
validation_preds = pd.read_parquet('example_validation_predictions.parquet')
spinner.succeed()


2021-09-09 23:58:24,069 INFO numerapi.utils: target file already exists
2021-09-09 23:58:24,071 INFO numerapi.utils: download complete
2021-09-09 23:58:24,941 INFO numerapi.utils: starting download
numerai_tournament_data_280.parquet: 100%|█████████▉| 582M/582M [00:34<00:00, 20.5MB/s]2021-09-09 23:59:00,300 INFO numerapi.utils: starting download

numerai_validation_data.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]
numerai_validation_data.parquet:   0%|          | 1.02k/228M [00:00<8:09:01, 7.75kB/s]
numerai_validation_data.parquet:   0%|          | 34.8k/228M [00:00<25:32, 148kB/s]   
numerai_validation_data.parquet:   0%|          | 86.0k/228M [00:00<15:13, 249kB/s]
numerai_validation_data.parquet:   0%|          | 174k/228M [00:00<09:28, 400kB/s] 
numerai_validation_data.parquet:   0%|          | 383k/228M [00:00<04:45, 795kB/s]
numerai_validation_data.parquet:   0%|          | 801k/228M [00:00<02:27, 1.54MB/s]
numerai_validation_data.parquet:   1%|          | 1.62M/228M [00:0

⠹ Reading parquet data

numerai_tournament_data_280.parquet: 582MB [00:56, 10.4MB/s]
numerai_validation_data.parquet: 228MB [00:20, 11.0MB/s]                           
example_predictions_280.parquet: 33.5MB [00:05, 5.79MB/s]                            
example_validation_predictions.parquet: 13.0MB [00:01, 6.61MB/s]                            


✔ Reading parquet data


In [41]:
targets = [t for t in training_data.columns if 'target' in t]
features = [f for f in training_data.columns if 'feature' in f]
eras = list(training_data['era'].unique())

In [43]:
training_data[targets]
values = dict()
for t in targets:
  values[t] = sorted(list(training_data[t].unique()))


values

{'target': [0.0, 0.25, 0.5, 0.75, 1.0],
 'target_alan_20': [0.0, 0.25, 0.5, 0.75, 1.0],
 'target_alan_60': [0.0, 0.25, 0.5, 0.75, 1.0, nan],
 'target_arthur_20': [0.0,
  0.16666667,
  0.33333334,
  0.5,
  0.6666667,
  0.8333333,
  1.0,
  nan],
 'target_arthur_60': [0.0,
  0.16666667,
  0.33333334,
  0.5,
  0.6666667,
  0.8333333,
  1.0,
  nan],
 'target_ben_20': [0.0, 0.25, 0.5, 0.75, 1.0],
 'target_ben_60': [0.0, 0.25, 0.5, 0.75, 1.0, nan],
 'target_george_20': [0.0, 0.25, 0.5, 0.75, 1.0],
 'target_george_60': [0.0, 0.25, 0.5, 0.75, 1.0, nan],
 'target_janet_20': [0.0, 0.25, 0.5, 0.75, 1.0, nan],
 'target_janet_60': [0.0, 0.25, 0.5, 0.75, 1.0, nan],
 'target_jerome_20': [0.0, 0.25, 0.5, 0.75, 1.0, nan],
 'target_jerome_60': [0.0, 0.25, 0.5, 0.75, 1.0, nan],
 'target_nomi_20': [0.0, 0.25, 0.5, 0.75, 1.0],
 'target_nomi_60': [0.0, 0.25, 0.5, 0.75, 1.0, nan],
 'target_paul_20': [0.0, 0.25, 0.5, 0.75, 1.0, nan],
 'target_paul_60': [0.0, 0.25, 0.5, 0.75, 1.0, nan],
 'target_thomas_20': [0.

In [44]:
training_data[targets].corr()

,target,target_nomi_20,target_nomi_60,target_jerome_20,target_jerome_60,target_janet_20,target_janet_60,target_ben_20,target_ben_60,target_alan_20,target_alan_60,target_paul_20,target_paul_60,target_george_20,target_george_60,target_william_20,target_william_60,target_arthur_20,target_arthur_60,target_thomas_20,target_thomas_60
target,1.000000,1.000000,0.503147,0.769365,0.455448,0.659012,0.392255,0.819540,0.455063,0.678464,0.382948,0.751827,0.441127,0.722665,0.401227,0.819724,0.484425,0.829850,0.490358,0.880602,0.486349
target_nomi_20,1.000000,1.000000,0.503147,0.769365,0.455448,0.659012,0.392255,0.819540,0.455063,0.678464,0.382948,0.751827,0.441127,0.722665,0.401227,0.819724,0.484425,0.829850,0.490358,0.880602,0.486349
target_nomi_60,0.503147,0.503147,1.000000,0.433388,0.817732,0.373979,0.689912,0.456754,0.813261,0.384636,0.673598,0.423820,0.791360,0.409423,0.705469,0.464266,0.873005,0.468459,0.879774,0.492558,0.877989
target_jerome_20,0.769365,0.769365,0.433388,1.000000,0.463756,0.698353,0.382269,0.768737,0.426895,0.628114,0.353125,0.905954,0.446691,0.689335,0.379908,0.865786,0.453824,0.871040,0.457756,0.748965,0.421251
target_jerome_60,0.455448,0.455448,0.817732,0.463756,1.000000,0.382460,0.693168,0.450810,0.821748,0.372656,0.659323,0.451384,0.884963,0.408582,0.719172,0.457941,0.862257,0.459407,0.865918,0.447718,0.789250
target_janet_20,0.659012,0.659012,0.373979,0.698353,0.382460,1.000000,0.422820,0.641273,0.358757,0.718364,0.392251,0.703169,0.370593,0.576267,0.318308,0.724738,0.385911,0.747625,0.395341,0.648114,0.364063
target_janet_60,0.392255,0.392255,0.689912,0.382269,0.693168,0.422820,1.000000,0.378579,0.672070,0.414573,0.769956,0.374259,0.695526,0.341069,0.596787,0.388464,0.720954,0.395379,0.742915,0.386067,0.677794
target_ben_20,0.819540,0.819540,0.456754,0.768737,0.450810,0.641273,0.378579,1.000000,0.485953,0.655087,0.373347,0.745765,0.433137,0.757761,0.418228,0.746513,0.445281,0.755789,0.449744,0.872847,0.476992
target_ben_60,0.455063,0.455063,0.813261,0.426895,0.821748,0.358757,0.672070,0.485953,1.000000,0.373185,0.646430,0.416077,0.786051,0.425653,0.747294,0.424105,0.787089,0.427728,0.795083,0.480622,0.867900
target_alan_20,0.678464,0.678464,0.384636,0.628114,0.372656,0.718364,0.414573,0.655087,0.373185,1.000000,0.419483,0.614438,0.361734,0.581200,0.328936,0.642762,0.381904,0.661255,0.392130,0.702886,0.385541
